<a href="https://colab.research.google.com/github/Joiljoe89/deep-learning-in-computer-vision/blob/master/Tool_contact_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connnect to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IMPORTS
We will use Keras (Tensorflow 2) for building our ResNet model and h5py to load data

In [42]:
import os
#import h5py
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline
import tensorflow.keras.backend as K
# Image dataset has channels as its last dimensions
K.set_image_data_format('channels_last')

LOAD DATA
Tool calibration Dataset

In [4]:
#For the given path, get the List of all files in the directory tree 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [28]:
# Load the dataset

img_c1 = cv2.imread('/content/drive/My Drive/Contact_check/train/noContact/frame0.jpg')
img_c1 = cv2.resize(img_c1, (224,224), interpolation = cv2.INTER_CUBIC)
img_c1 = np.array(img_c1)

#img_c3 = cv2.imread('/home/joe/Desktop/sgan/train/patch/train_data/grass/ocn10.jpg')

input_img = 0

# training images

dirName_c1 = '/content/drive/My Drive/Contact_check/train/noContact';
dirName_c2 = '/content/drive/My Drive/Contact_check/train/Contact';
#dirName_c3 = '/home/joe/Desktop/sgan/train/patch/train_data/grass';

# Get the list of all files in directory tree at given path
listOfFiles_c1 = getListOfFiles(dirName_c1)
listOfFiles_c1 = sorted(listOfFiles_c1)
listOfFiles_c2 = getListOfFiles(dirName_c2)
listOfFiles_c2 = sorted(listOfFiles_c2)
#listOfFiles_c3 = getListOfFiles(dirName_c3)
#listOfFiles_c3 = sorted(listOfFiles_c3)

print('total training nonContact images:', len(os.listdir(dirName_c1)))
num_img_c1 = len(os.listdir(dirName_c1))
print('total training Contact images:', len(os.listdir(dirName_c2)))
num_img_c2 = len(os.listdir(dirName_c2))

# Print the files
for elem_c1 in listOfFiles_c1:
  #print(elem_c1[54::])
  img_in = cv2.imread('/content/drive/My Drive/Contact_check/train/noContact/'+elem_c1[54::])
  img_in = cv2.resize(img_in, (224,224), interpolation = cv2.INTER_CUBIC) #INTER_NEAREST,INTER_LINEAR,INTER_CUBIC,INTER_LANCZOS4 
  img_in =np.array(img_in)
  #print(img_c1.shape)
  input_img = np.append(img_in,img_c1, axis=0)
  img_c1 = input_img
print(img_c1.shape)

for elem_c2 in listOfFiles_c2:
  #print(elem_c2[52::])
  img_in = cv2.imread('/content/drive/My Drive/Contact_check/train/Contact/'+elem_c2[52::])
  img_in = cv2.resize(img_in, (224,224), interpolation = cv2.INTER_CUBIC)
  img_in =np.array(img_in)
  input_img = np.append(img_c1,img_in, axis=0)
  img_c1 = input_img
print(img_c1.shape)
img_c1 = img_c1.reshape(1+num_img_c1+num_img_c2,224,224,3)

input_img = 0


total training nonContact images: 415
total training Contact images: 34
(93184, 224, 3)
(100800, 224, 3)


In [52]:
input_img = 0
#print(img_c1.shape,img_c2.shape,img_c3.shape,img_c4.shape,img_c5.shape)
#nb_img_c1 = int(img_c1.shape[0]/200)

max_value = 255.0
X_train = img_c1.astype('float32') / max_value
#img_c2 =img_c2.astype('float32') / max_value

#X_train = X_train.reshape(img_c1[0] + img_c2[0],224,224,3)
print('total data:', X_train.shape)

total data: (450, 224, 224, 3)


In [66]:
y_train = np.append(np.zeros((num_img_c1+1,2)),np.ones((num_img_c2,2)), axis=0)
#y_train = np.append(y_train,2*np.ones(nb_img_c3))
Y_train = y_train#.reshape(X_train[0], 2)
print('total label',Y_train.shape)


print ("********************train data complete****************")

total label (450, 2)
********************train data complete****************


IDENTITY BLOCK

A block of layers with skip connection, where the input activation (say a[i]) has the same dimension as the output activation (say a[i+n] where n is number of layers in the block) is an identity block in a ResNet. The skip-connection or shortcut is reffered to the path 1 in the figure. While path 2 is the main path.

Structure of our identity block

In [44]:
def identity_block(X, f, filters, stage, block):

  '''
  Implementation of identity block described above

  Arguments:
  X -       input tensor to the block of shape (m, n_H_prev, n_W_prev, n_C_prev)
  f -       defines shpae of filter in the middle layer of the main path
  filters - list of integers, defining the number of filters in each layer of the main path
  stage -   defines the block position in the network
  block -   used for naming convention

  Returns: 
  X - output is a tensor of shape (n_H, n_W, n_C) which matches (m, n_H_prev, n_W_prev, n_C_prev)
  '''

  # defining base name for block
  conv_base_name = 'res' + str(stage) + block + '_'
  bn_base_name = 'bn' + str(stage) + block + '_'

  # retrieve number of filters in each layer of main path
  # NOTE: f3 must be equal to n_C. That way dimensions of the third component will match the dimension of original input to identity block
  f1, f2, f3 = filters

  # Batch normalization must be performed on the 'channels' axis for input. It is 3, for our case
  bn_axis = 3

  # save input for "addition" to last layer output; step in skip-connection
  X_skip_connection = X

  # ----------------------------------------------------------------------
  # Building layers/component of identity block using Keras functional API

  # First component/layer of main path
  X = Conv2D(filters= f1, kernel_size = (1,1), strides = (1,1), padding='valid', name=conv_base_name+'first_component', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis, name=bn_base_name+'first_component')(X)
  X = Activation('relu')(X)
  
  # Second component/layer of main path
  X = Conv2D(filters= f2, kernel_size = (f,f), strides = (1,1), padding='same', name=conv_base_name+'second_component', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis, name=bn_base_name+'second_component')(X)
  X = Activation('relu')(X)

  # Third component/layer of main path
  X = Conv2D(filters= f3, kernel_size = (1,1), strides = (1,1), padding='valid', name=conv_base_name+'third_component', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis, name=bn_base_name+'third_component')(X)
  
  # "Addition step" - skip-connection value merges with main path
  # NOTE: both values have same dimensions at this point, so no operation is required to match dimensions
  X = Add()([X, X_skip_connection])
  X = Activation('relu')(X)

  return X

CONVOLUTIONAL BLOCK

This is another kind of block with skip-connection in the ResNet. In this the input to block, and output of last layer in the block don't match in dimensions. So we need to add a convolution step in the skip-connection, before adding the input to last layer's output. This convolution step changes the dimension of the input, so that the dimensions match up for the later addition step.

In [45]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in above figure
    
    Arguments:
    X -       input tensor to the block of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -       defines shape of filter in the middle layer of the main path
    filters - list of integers, defining the number of filters in each layer of the main path
    stage -   defines the block position in the network
    block -   used for naming convention
    s -       specifies the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining base name for block
    conv_base_name = 'res' + str(stage) + block + '_'
    bn_base_name = 'bn' + str(stage) + block + '_'
    
    # retrieve number of filters in each layer of main path
    f1, f2, f3 = filters
    
    # Batch normalization must be performed on the 'channels' axis for input. It is 3, for our case
    bn_axis = 3

    # save input for "addition" to last layer output; step in skip-connection
    X_skip_connection = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(f1, (1, 1), strides = (s,s), padding = 'valid', name = conv_base_name + 'first_component', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = bn_axis, name = bn_base_name + 'first_component')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(f2,  kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_base_name + 'second_component', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = bn_axis, name = bn_base_name + 'second_component')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(f3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_base_name + 'third_component', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = bn_axis, name = bn_base_name + 'third_component')(X)

    ##### Convolve skip-connection value to match its dimensions to third layer output's dimensions #### 
    X_skip_connection = Conv2D(f3, (1, 1), strides = (s,s), padding = 'valid', name = conv_base_name + 'merge', kernel_initializer = glorot_uniform(seed=0))(X_skip_connection)
    X_skip_connection = BatchNormalization(axis = 3, name = bn_base_name + 'merge')(X_skip_connection)

    # "Addition step" 
    # NOTE: both values have same dimensions at this point
    X = Add()([X, X_skip_connection])
    X = Activation('relu')(X)

    return X

RESNET50

Now that we have our building blocks - Convolutional block and identity block in place, we will build a 50 layer deep neural network with skip connections that implements the follwoing architecture

In [68]:
def ResNet50(input_shape = (64, 64, 3), classes = 2):
    """
    Arguments:
    input_shape - shape of the images of the dataset
    classes - number of classes

    Returns:
    model - a Model() instance in Keras
    
    """
    
    # plug in input_shape to define the input tensor
    X_input = Input(input_shape)

    # Zero-Padding : pads the input with a pad of (3,3)
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv_1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # NOTE: dimensions of filters that are passed to identity block are such that final layer output
    # in identity block mathces the original input to the block
    # blocks in each stage are alphabetically sequenced

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Average Pooling
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [69]:
model = ResNet50(input_shape = (224, 224, 3), classes = 2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

TRAIN RESNET50

In [72]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 16)

Epoch 1/2
29/29 [==============================] - 284s 10s/step - loss: 0.1315 - accuracy: 0.4444
Epoch 2/2
29/29 [==============================] - 278s 10s/step - loss: 0.1208 - accuracy: 0.4711


In [ ]:
model.save('/content/drive/MyDrive/ResNet50/model/RESNET50.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


TEST PERFORMANCE

In [73]:
predictions = model.evaluate(X_train, Y_train)
print("Loss = " + str(predictions[0]))
print("Test Accuracy = " + str(predictions[1]))

15/15 [==============================] - 61s 4s/step - loss: 0.5092 - accuracy: 0.8911
Loss = 0.5091859698295593
Test Accuracy = 0.8911111354827881


In [ ]:
new_model = load_model('/content/drive/MyDrive/ResNet50/model/RESNET50.h5')
predictions = new_model.evaluate(X_test, Y_test)
print(predictions)

4/4 [==============================] - 3s 521ms/step - loss: 0.2398 - accuracy: 0.9500
[0.239801824092865, 0.949999988079071]
